In [78]:
import numpy as np
import pandas as pd
from datetime import datetime

import seaborn as sns
import matplotlib.pyplot as plt
sns.set_style("whitegrid")
%matplotlib inline

from datetime import datetime
import time
import csv
sns.set_style("darkgrid")
results_2016 = pd.read_csv('./Runners_all_results_2016.csv', header = 0, encoding='latin-1')
results_2016['OfficialTime'] = results_2016['OfficialTime'].apply(lambda x: datetime.combine(datetime.min, datetime.strptime(x, '%H:%M:%S').time()) - datetime.min)
results_2017 = pd.read_csv('./Runners_all_results_2017.csv', header = 0, encoding='latin-1')
results_2017['OfficialTime'] = results_2017['OfficialTime'].apply(lambda x: datetime.combine(datetime.min, datetime.strptime(x, '%H:%M:%S').time()) - datetime.min)
results_2018 = pd.read_csv('./Runners_all_results_2018.csv', header = 0, encoding='latin-1')
results_2018['OfficialTime'] = results_2018['OfficialTime'].apply(lambda x: datetime.combine(datetime.min, datetime.strptime(x, '%H:%M:%S').time()) - datetime.min)
results_2019 = pd.read_csv('./Runners_all_results_2019.csv', header = 0, encoding='latin-1', error_bad_lines=False)
results_2019['OfficialTime'] = results_2019['OfficialTime'].apply(lambda x: datetime.combine(datetime.min, datetime.strptime(x, '%H:%M:%S').time()) - datetime.min)

b'Skipping line 26144: expected 20 fields, saw 21\n'


In [79]:
results_2016['Year']=2016
results_2017['Year']=2017
results_2018['Year']=2018
results_2019['Year']=2019
results = results_2019.append(results_2018, ignore_index = True, sort=True).append(results_2017, ignore_index = True, sort=True).append(results_2016, ignore_index = True, sort=True)


In [80]:
import requests
import pandas as pd
import geocoder

# script for returning elevation from lat, long
def get_elevation(lat, long):
    query = ('https://elevation-api.io/api/elevation'
             f'?points=({lat},{long})')
    r = requests.get(query, headers = {'User-agent': 'your bot 0.1'})
    if  r.status_code == requests.codes.ok:
        rj = r.json()# json object, various ways you can extract value
        # one approach is to use pandas json functionality:
        elevation = pd.io.json.json_normalize(rj, 'elevations')['elevation'].values[0]
        return elevation
    else:
        print("response status code: {}".format(r.status_code))
        return

def get_elevation_from_name(name):
    g = geocoder.arcgis(name)
    # print(g.lat, g.lng)
    elevation = get_elevation(g.lat, g.lng)
    # print(elevation)
    return elevation

In [81]:
# cities = results_2019[['City']]
# cities.drop_duplicates(inplace = True)
# cities['Elevation'] = cities.apply(lambda x: get_elevation_from_name(x['City']), axis=1)
# cities.to_csv("cities.csv", encoding='utf-8', index=False)

In [92]:
cities_elevation = pd.read_csv('cities.csv', header = 0, sep=' *, *', quotechar = '"', error_bad_lines=False)
cities_elevation['Elevation'] = cities_elevation['Elevation'].apply(pd.to_numeric, errors='coerce')
results = results.merge(cities_elevation.set_index("City"), left_on='City', right_on="City", how='left')

/home/micha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.
Skipping line 286: Expected 2 fields in line 286, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 968: Expected 2 fields in line 968, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 1052: Expected 2 fields in line 1052, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 1186: Expected 2 fields in line 1186, saw 3. Error could possibly be due to quotes being ignored when a multi-char delimiter is used.
Skipping line 1276: Expected 2 fields in line 12

,City,Elevation
0,Eldoret,2093.5
1,Ambo,2940.0
2,Iten,2083.5
3,Keringet,1955.0
4,Kaptagat,2424.5
...,...,...
11731,North Granby,177.0
11732,Elmora,608.0
11733,APO,921.5
11734,Spotswood,15.0


In [88]:
cities = results[['City','Elevation']]
cities.drop_duplicates(inplace = True)

/home/micha/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


(12234, 2)

In [84]:
filter = cities['Elevation'].isnull()
cities = cities[filter]

,City,Elevation
389,"Boston, Ma",NaN
1758,"Zibo City, Shandong Province",NaN
1940,"Tokyo-To, Chiyoda-Ku",NaN
2249,"Nt, Hk",NaN
2455,"Santacruz (east), Mumbai",NaN
...,...,...
113022,"Roodepoort, Johannesburg",NaN
113152,"Hangzhou, Zhejiang",NaN
113159,"Redfern, Sydney",NaN
113179,"Bronx,",NaN


In [91]:
# cities['Elevation'] = cities.apply(lambda x: get_elevation_from_name(x['City']), axis=1)
cities.to_csv("cities11.csv", encoding='utf-8', index=False)

In [89]:
cities

,City,Elevation
0,Eldoret,2093.5
3,Ambo,2940.0
7,Iten,2083.5
8,Keringet,1955.0
9,Kaptagat,2424.5
...,...,...
114337,North Granby,177.0
114364,Elmora,608.0
114381,APO,921.5
114424,Spotswood,15.0
